# 📘 Day 3: Advanced ML Techniques (Feature Engineering, Stacking, Pipelines)

**🎯 Goal:** Master advanced techniques for building production-ready ML systems

**⏱️ Time:** 90-120 minutes

**🌟 Why This Matters for AI:**
- Feature engineering often provides bigger gains than algorithm choice
- ML pipelines are essential for production deployment
- Stacking/blending techniques power top Kaggle solutions
- Critical for building reliable Agentic AI systems
- Required knowledge for ML engineering roles in 2024-2025
- Foundation for combining traditional ML with modern neural networks

---

## 🎯 What You'll Learn Today

**1. Feature Engineering**
- Creating powerful features from raw data
- Feature selection and importance
- Handling missing values and outliers

**2. Model Stacking & Blending**
- Combining multiple models intelligently
- Stacked generalization
- Kaggle-winning techniques

**3. ML Pipelines**
- Building reproducible workflows
- Preventing data leakage
- Production deployment patterns

**4. Real AI Example**
- Building production ML pipeline for Agentic AI
- End-to-end workflow

Let's become ML engineers! 🚀

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE

# ML Pipeline components
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

# Models for stacking
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    StackingClassifier,
    VotingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

# Make plots beautiful
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("Let's build production ML systems! 🚀")

## 📊 Our Dataset: User Intent Classification for Agentic AI

**Real AI Application:** Building an intent classifier for an AI agent system

**Scenario:** You're building an Agentic AI that routes user requests to specialized agents:
- **Customer Support Agent**: Handles complaints, questions
- **Sales Agent**: Handles purchases, pricing
- **Technical Agent**: Handles bugs, API issues
- **General Agent**: Everything else

**Challenge:** Classify user intent from message features

**Features:**
- `message_length`: Number of words
- `has_question_mark`: Contains '?'
- `urgent_keywords`: Contains urgent words ("ASAP", "urgent")
- `technical_keywords`: Contains tech terms
- `sentiment_score`: Negative to positive (-1 to 1)
- `time_of_day`: Hour message sent
- `user_history_count`: Previous messages

**Target:**
- `intent`: 0=General, 1=Support, 2=Sales, 3=Technical

In [ ]:
# Create realistic intent classification dataset
np.random.seed(42)

n_samples = 3000
samples_per_class = n_samples // 4

def create_class_data(class_id):
    """Generate features for each intent class"""
    if class_id == 0:  # General
        return {
            'message_length': np.random.poisson(15, samples_per_class),
            'has_question_mark': np.random.binomial(1, 0.3, samples_per_class),
            'urgent_keywords': np.random.binomial(1, 0.1, samples_per_class),
            'technical_keywords': np.random.binomial(1, 0.2, samples_per_class),
            'sentiment_score': np.random.normal(0, 0.3, samples_per_class),
            'time_of_day': np.random.choice(24, samples_per_class),
            'user_history_count': np.random.poisson(5, samples_per_class)
        }
    elif class_id == 1:  # Support
        return {
            'message_length': np.random.poisson(25, samples_per_class),
            'has_question_mark': np.random.binomial(1, 0.7, samples_per_class),
            'urgent_keywords': np.random.binomial(1, 0.5, samples_per_class),
            'technical_keywords': np.random.binomial(1, 0.3, samples_per_class),
            'sentiment_score': np.random.normal(-0.3, 0.4, samples_per_class),
            'time_of_day': np.random.choice(24, samples_per_class),
            'user_history_count': np.random.poisson(3, samples_per_class)
        }
    elif class_id == 2:  # Sales
        return {
            'message_length': np.random.poisson(20, samples_per_class),
            'has_question_mark': np.random.binomial(1, 0.5, samples_per_class),
            'urgent_keywords': np.random.binomial(1, 0.2, samples_per_class),
            'technical_keywords': np.random.binomial(1, 0.1, samples_per_class),
            'sentiment_score': np.random.normal(0.2, 0.3, samples_per_class),
            'time_of_day': np.random.choice(range(9, 18), samples_per_class),  # Business hours
            'user_history_count': np.random.poisson(2, samples_per_class)
        }
    else:  # Technical
        return {
            'message_length': np.random.poisson(30, samples_per_class),
            'has_question_mark': np.random.binomial(1, 0.4, samples_per_class),
            'urgent_keywords': np.random.binomial(1, 0.4, samples_per_class),
            'technical_keywords': np.random.binomial(1, 0.9, samples_per_class),
            'sentiment_score': np.random.normal(-0.1, 0.3, samples_per_class),
            'time_of_day': np.random.choice(24, samples_per_class),
            'user_history_count': np.random.poisson(8, samples_per_class)
        }

# Create data for all classes
all_data = []
for class_id in range(4):
    class_data = create_class_data(class_id)
    class_data['intent'] = [class_id] * samples_per_class
    all_data.append(pd.DataFrame(class_data))

# Combine and shuffle
df = pd.concat(all_data, ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

intent_names = {0: 'General', 1: 'Support', 2: 'Sales', 3: 'Technical'}

print("🤖 Agentic AI Intent Classification Dataset Created!")
print(f"Total messages: {len(df):,}")
print("\nIntent distribution:")
for intent_id, count in df['intent'].value_counts().sort_index().items():
    print(f"  {intent_names[intent_id]}: {count}")
print("\nFirst few messages:")
df.head()

## 🛠️ Part 1: Feature Engineering

**What is Feature Engineering?**
- Creating new features from existing ones
- Often more impactful than choosing the "best" algorithm!

**Common Techniques:**
1. **Polynomial features**: Create interactions (e.g., feature1 × feature2)
2. **Domain knowledge**: Engineer features based on problem understanding
3. **Aggregations**: Mean, max, count, etc.
4. **Binning**: Convert continuous to categorical

**🎯 Real AI Impact:**
- "Feature engineering is the key to winning Kaggle competitions" - Kaggle Grandmasters
- At Google/Facebook: Teams spend 80% time on features, 20% on algorithms

In [ ]:
# Feature Engineering: Create powerful new features
print("🔧 Engineering new features...\n")

df_engineered = df.copy()

# 1. Interaction features
df_engineered['urgent_technical'] = df['urgent_keywords'] * df['technical_keywords']
df_engineered['question_length'] = df['has_question_mark'] * df['message_length']

# 2. Ratio features
df_engineered['keywords_per_word'] = (df['urgent_keywords'] + df['technical_keywords']) / (df['message_length'] + 1)

# 3. Binning continuous features
df_engineered['message_length_bin'] = pd.cut(df['message_length'], bins=[0, 10, 20, 30, 100], labels=[0, 1, 2, 3])
df_engineered['message_length_bin'] = df_engineered['message_length_bin'].astype(int)

# 4. Time-based features
df_engineered['is_business_hours'] = ((df['time_of_day'] >= 9) & (df['time_of_day'] <= 17)).astype(int)
df_engineered['is_night'] = ((df['time_of_day'] >= 22) | (df['time_of_day'] <= 6)).astype(int)

# 5. User behavior features
df_engineered['is_new_user'] = (df['user_history_count'] <= 2).astype(int)
df_engineered['is_power_user'] = (df['user_history_count'] >= 10).astype(int)

# 6. Sentiment features
df_engineered['is_negative'] = (df['sentiment_score'] < -0.2).astype(int)
df_engineered['is_positive'] = (df['sentiment_score'] > 0.2).astype(int)

print("✅ Engineered Features Created:")
new_features = [col for col in df_engineered.columns if col not in df.columns]
for feat in new_features:
    print(f"  • {feat}")

print(f"\nOriginal features: {len(df.columns) - 1}")
print(f"Total features now: {len(df_engineered.columns) - 1}")
print(f"New features added: {len(new_features)}")

In [ ]:
# Compare model performance with and without engineered features
print("📊 Comparing: Original vs Engineered Features\n")

# Prepare both datasets
X_original = df.drop('intent', axis=1)
X_engineered = df_engineered.drop('intent', axis=1)
y = df['intent']

# Split
X_orig_train, X_orig_test, y_train, y_test = train_test_split(
    X_original, y, test_size=0.2, random_state=42, stratify=y
)
X_eng_train, X_eng_test, _, _ = train_test_split(
    X_engineered, y, test_size=0.2, random_state=42, stratify=y
)

# Train models
rf_original = RandomForestClassifier(n_estimators=100, random_state=42)
rf_original.fit(X_orig_train, y_train)
acc_original = accuracy_score(y_test, rf_original.predict(X_orig_test))

rf_engineered = RandomForestClassifier(n_estimators=100, random_state=42)
rf_engineered.fit(X_eng_train, y_train)
acc_engineered = accuracy_score(y_test, rf_engineered.predict(X_eng_test))

print(f"Original Features Accuracy: {acc_original:.2%}")
print(f"Engineered Features Accuracy: {acc_engineered:.2%}")
print(f"\n📈 Improvement: {(acc_engineered - acc_original):.2%}")

if acc_engineered > acc_original:
    print("\n🎉 Feature engineering improved our model!")
else:
    print("\n💡 Original features were already quite good!")

## 🎯 Feature Selection: Choosing the Best Features

**Why Feature Selection?**
- Too many features → Overfitting, slower training
- Remove irrelevant/redundant features
- Simpler models → Better production performance

**Methods:**
1. **Filter methods**: Statistical tests (fast)
2. **Wrapper methods**: Try feature subsets (accurate, slow)
3. **Embedded methods**: Feature importance from models

Let's try all three!

In [ ]:
# Method 1: Filter method (SelectKBest)
print("🔍 Feature Selection Methods\n")
print("1️⃣ Filter Method: SelectKBest\n")

selector = SelectKBest(f_classif, k=10)
X_selected = selector.fit_transform(X_eng_train, y_train)

# Get selected feature names
selected_features = X_eng_train.columns[selector.get_support()].tolist()
print("Top 10 features by statistical test:")
for i, feat in enumerate(selected_features, 1):
    print(f"  {i}. {feat}")

# Method 2: Embedded method (Random Forest importance)
print("\n2️⃣ Embedded Method: Random Forest Feature Importance\n")

feature_importance = pd.DataFrame({
    'Feature': X_eng_train.columns,
    'Importance': rf_engineered.feature_importances_
}).sort_values('Importance', ascending=False).head(10)

print("Top 10 features by Random Forest:")
print(feature_importance.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='steelblue')
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('🎯 Top 10 Most Important Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🏗️ Part 2: Model Stacking & Blending

**What is Stacking?**
- Train multiple diverse models (Level 0)
- Use their predictions as features for another model (Level 1)
- Level 1 model learns how to best combine Level 0 predictions

**Stacking vs Voting:**
- **Voting**: Simple average or majority vote
- **Stacking**: Learns optimal way to combine models

**Why Stacking Works:**
- Different models make different errors
- Meta-model learns which model to trust when
- Often 1-2% improvement (huge in competitions!)

**🎯 Real AI Use Cases:**
- **Kaggle**: Almost all top solutions use stacking
- **Production**: Combine traditional ML + neural networks
- **Netflix Prize**: Winning solution stacked 100+ models
- **Modern RAG**: Stack retrieval scores from multiple sources

In [ ]:
# Build stacked ensemble
print("🏗️ Building Stacked Ensemble\n")

# Level 0 models (base models) - diverse algorithms
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss')),
    ('lgb', lgb.LGBMClassifier(n_estimators=100, random_state=42, verbose=-1))
]

# Level 1 model (meta-model)
meta_model = LogisticRegression(multi_class='multinomial', random_state=42, max_iter=1000)

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5  # 5-fold cross-validation to avoid overfitting
)

print("Training stacked ensemble...")
stacking_clf.fit(X_eng_train, y_train)

# Predictions
y_pred_stack = stacking_clf.predict(X_eng_test)
acc_stack = accuracy_score(y_test, y_pred_stack)

print(f"\n✅ Stacking Ensemble Accuracy: {acc_stack:.2%}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_stack, target_names=list(intent_names.values())))

In [ ]:
# Compare individual models vs stacking
print("📊 Comparing Individual Models vs Stacking\n")

# Train individual models
results = []

for name, model in base_models:
    model.fit(X_eng_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_eng_test))
    results.append({'Model': name.upper(), 'Accuracy': acc})

results.append({'Model': 'STACKING', 'Accuracy': acc_stack})

results_df = pd.DataFrame(results).sort_values('Accuracy', ascending=False)
print(results_df.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
bars = plt.bar(results_df['Model'], results_df['Accuracy'], color=colors)
plt.ylabel('Accuracy', fontsize=12)
plt.title('🏆 Individual Models vs Stacking Ensemble', fontsize=14, fontweight='bold')
plt.ylim(0.7, 1.0)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2%}',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

if results_df.iloc[0]['Model'] == 'STACKING':
    print("\n🎉 Stacking beats all individual models!")
else:
    print("\n💡 Stacking performs competitively with the best individual model!")

## 🔧 Part 3: ML Pipelines

**What is an ML Pipeline?**
- Automates the entire ML workflow
- Preprocessing → Feature Engineering → Model Training
- Ensures reproducibility and prevents data leakage

**Why Pipelines Matter:**
- ✅ **Prevent data leakage**: Fit only on training data
- ✅ **Reproducible**: Same steps every time
- ✅ **Production-ready**: Easy to deploy
- ✅ **Clean code**: No manual step tracking

**🎯 Real AI Production:**
- Every production ML system uses pipelines
- Required for MLOps and model monitoring
- Essential for A/B testing new models
- Critical for Agentic AI systems

In [ ]:
# Build a basic ML pipeline
print("🔧 Building ML Pipeline\n")

# Define pipeline steps
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(f_classif, k=10)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

print("Pipeline Steps:")
for i, (name, step) in enumerate(pipeline.steps, 1):
    print(f"  {i}. {name}: {step.__class__.__name__}")

# Train pipeline (all steps executed automatically)
print("\nTraining pipeline...")
pipeline.fit(X_eng_train, y_train)

# Predict (preprocessing applied automatically)
y_pred_pipeline = pipeline.predict(X_eng_test)
acc_pipeline = accuracy_score(y_test, y_pred_pipeline)

print(f"\n✅ Pipeline Accuracy: {acc_pipeline:.2%}")
print("\n💡 Pipeline automatically applied all transformations to test data!")

In [ ]:
# Advanced pipeline with custom transformations
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Custom transformer for feature engineering"""
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        
        # Add engineered features
        X_new['urgent_technical'] = X['urgent_keywords'] * X['technical_keywords']
        X_new['question_length'] = X['has_question_mark'] * X['message_length']
        X_new['is_business_hours'] = ((X['time_of_day'] >= 9) & (X['time_of_day'] <= 17)).astype(int)
        X_new['is_negative'] = (X['sentiment_score'] < -0.2).astype(int)
        
        return X_new

# Build advanced pipeline
advanced_pipeline = Pipeline([
    ('feature_engineering', FeatureEngineer()),
    ('scaler', StandardScaler()),
    ('classifier', xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss'))
])

print("🚀 Advanced Pipeline with Custom Feature Engineering\n")
print("Pipeline Steps:")
for i, (name, step) in enumerate(advanced_pipeline.steps, 1):
    print(f"  {i}. {name}: {step.__class__.__name__}")

# Train
advanced_pipeline.fit(X_orig_train, y_train)

# Predict
y_pred_adv = advanced_pipeline.predict(X_orig_test)
acc_adv = accuracy_score(y_test, y_pred_adv)

print(f"\n✅ Advanced Pipeline Accuracy: {acc_adv:.2%}")
print("\n💡 This pipeline is production-ready! Can be saved and deployed.")

## 🌟 Real AI Example: Production ML Pipeline for Agentic AI

**Full Production Pipeline:**
1. Feature engineering
2. Preprocessing and scaling
3. Feature selection
4. Stacked ensemble
5. Model versioning and monitoring

**This is what you'd deploy in a real Agentic AI system!**

In [ ]:
# Production-ready pipeline for Agentic AI
print("🤖 Building Production ML Pipeline for Agentic AI\n")
print("="*60)

# Step 1: Feature Engineering Pipeline
feature_pipeline = Pipeline([
    ('engineer', FeatureEngineer()),
    ('scaler', StandardScaler())
])

# Step 2: Prepare data
X_processed_train = feature_pipeline.fit_transform(X_orig_train)
X_processed_test = feature_pipeline.transform(X_orig_test)

# Step 3: Build stacked ensemble with diverse models
production_base_models = [
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)),
    ('xgb', xgb.XGBClassifier(n_estimators=200, learning_rate=0.1, random_state=42, eval_metric='mlogloss')),
    ('lgb', lgb.LGBMClassifier(n_estimators=200, learning_rate=0.1, random_state=42, verbose=-1))
]

production_meta_model = xgb.XGBClassifier(
    n_estimators=50, 
    learning_rate=0.05, 
    random_state=42,
    eval_metric='mlogloss'
)

production_model = StackingClassifier(
    estimators=production_base_models,
    final_estimator=production_meta_model,
    cv=5
)

print("Training production model...")
print("  Base Models: Random Forest, XGBoost, LightGBM")
print("  Meta Model: XGBoost")
print("  Cross-Validation: 5-fold")
print("\nThis may take a moment...\n")

production_model.fit(X_processed_train, y_train)

# Step 4: Evaluate
y_pred_prod = production_model.predict(X_processed_test)
acc_prod = accuracy_score(y_test, y_pred_prod)

print("="*60)
print("\n🎯 PRODUCTION MODEL RESULTS\n")
print(f"Overall Accuracy: {acc_prod:.2%}")
print("\n📊 Detailed Classification Report:")
print(classification_report(y_test, y_pred_prod, target_names=list(intent_names.values())))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_prod)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(intent_names.values()),
            yticklabels=list(intent_names.values()))
plt.title('🎯 Production Model: Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('Actual Intent', fontsize=12)
plt.xlabel('Predicted Intent', fontsize=12)
plt.tight_layout()
plt.show()

print("\n✅ Production pipeline is ready for deployment!")

## 🚀 Deployment Simulation: How This Works in Production

**Real-world deployment flow:**

1. **User sends message** → "Help! My API is returning 500 errors"
2. **Extract features** → message_length=7, has_question_mark=0, technical_keywords=1, etc.
3. **Pipeline processes** → Feature engineering, scaling, prediction
4. **Route to agent** → Technical Agent
5. **Agent responds** → "I'll help with your API error..."

Let's simulate this!

In [ ]:
# Simulate production inference
print("🤖 PRODUCTION DEPLOYMENT SIMULATION\n")
print("="*60)

# Simulate new incoming messages
new_messages = pd.DataFrame([
    {
        'message_length': 7,
        'has_question_mark': 0,
        'urgent_keywords': 1,
        'technical_keywords': 1,
        'sentiment_score': -0.4,
        'time_of_day': 14,
        'user_history_count': 5,
        'description': 'API returning 500 errors'
    },
    {
        'message_length': 12,
        'has_question_mark': 1,
        'urgent_keywords': 0,
        'technical_keywords': 0,
        'sentiment_score': 0.3,
        'time_of_day': 11,
        'user_history_count': 1,
        'description': 'How much does the pro plan cost?'
    },
    {
        'message_length': 20,
        'has_question_mark': 1,
        'urgent_keywords': 1,
        'technical_keywords': 0,
        'sentiment_score': -0.5,
        'time_of_day': 16,
        'user_history_count': 3,
        'description': 'URGENT: Need refund, service not working!'
    },
    {
        'message_length': 8,
        'has_question_mark': 1,
        'urgent_keywords': 0,
        'technical_keywords': 0,
        'sentiment_score': 0.1,
        'time_of_day': 10,
        'user_history_count': 2,
        'description': 'What are your business hours?'
    }
])

# Extract features for prediction
new_messages_features = new_messages.drop('description', axis=1)

# Apply production pipeline
new_messages_processed = feature_pipeline.transform(new_messages_features)
predictions = production_model.predict(new_messages_processed)
prediction_proba = production_model.predict_proba(new_messages_processed)

# Display results
print("\n📨 INCOMING MESSAGES & ROUTING DECISIONS:\n")

for i, (idx, msg) in enumerate(new_messages.iterrows()):
    predicted_intent = intent_names[predictions[i]]
    confidence = prediction_proba[i][predictions[i]]
    
    print(f"Message {i+1}: \"{msg['description']}\"")
    print(f"  → Routed to: {predicted_intent} Agent")
    print(f"  → Confidence: {confidence:.1%}")
    print()

print("="*60)
print("\n✅ Production system successfully routing messages to correct agents!")
print("\n💡 In production, this would:")
print("  1. Run in < 10ms for real-time routing")
print("  2. Log all predictions for monitoring")
print("  3. Track accuracy and retrain periodically")
print("  4. Handle thousands of requests per second")

## 🎯 YOUR TURN: Exercise - Build Your Own Production Pipeline

**Challenge:** Create a complete production pipeline for a new problem!

**Problem:** Predict whether a support ticket needs escalation

**Your Task:**
1. Engineer at least 3 new features
2. Build a pipeline with preprocessing + model
3. Use cross-validation to evaluate
4. Report accuracy

Use the dataset below. Good luck! 💪

In [ ]:
# Ticket escalation dataset
np.random.seed(42)

n_tickets = 2000

ticket_data = {
    'response_time_hours': np.concatenate([
        np.random.exponential(24, 600),   # Escalated: slow response
        np.random.exponential(4, 1400)    # Not escalated: fast response
    ]),
    'num_messages': np.concatenate([
        np.random.poisson(10, 600),
        np.random.poisson(3, 1400)
    ]),
    'customer_satisfaction': np.concatenate([
        np.random.beta(2, 5, 600) * 5,    # Escalated: low satisfaction
        np.random.beta(5, 2, 1400) * 5    # Not escalated: high satisfaction
    ]),
    'priority_level': np.concatenate([
        np.random.choice([1, 2, 3], 600, p=[0.6, 0.3, 0.1]),
        np.random.choice([1, 2, 3], 1400, p=[0.1, 0.3, 0.6])
    ]),
    'needs_escalation': [1] * 600 + [0] * 1400
}

ticket_df = pd.DataFrame(ticket_data)
ticket_df = ticket_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("🎫 Support Ticket Escalation Dataset:")
print(ticket_df.head())
print(f"\nTotal tickets: {len(ticket_df)}")
print(f"Needs escalation: {ticket_df['needs_escalation'].sum()}")
print(f"Escalation rate: {ticket_df['needs_escalation'].mean():.1%}")

In [ ]:
# YOUR CODE HERE!
# Build a production pipeline for ticket escalation prediction

# Step 1: Engineer features
# YOUR CODE

# Step 2: Create pipeline
# YOUR CODE

# Step 3: Train and evaluate
# YOUR CODE

<details>
<summary>📖 Click here for solution</summary>

```python
# Step 1: Feature Engineering
ticket_df['messages_per_hour'] = ticket_df['num_messages'] / (ticket_df['response_time_hours'] + 1)
ticket_df['is_high_priority'] = (ticket_df['priority_level'] == 1).astype(int)
ticket_df['is_unsatisfied'] = (ticket_df['customer_satisfaction'] < 2.5).astype(int)
ticket_df['slow_response'] = (ticket_df['response_time_hours'] > 12).astype(int)

# Step 2: Prepare data
X = ticket_df.drop('needs_escalation', axis=1)
y = ticket_df['needs_escalation']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Build pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Step 4: Train and evaluate
pipeline.fit(X_train, y_train)
accuracy = accuracy_score(y_test, pipeline.predict(X_test))
print(f"Accuracy: {accuracy:.2%}")
```
</details>

## 🎓 Key Takeaways

**You just mastered advanced ML techniques!**

### **1. Feature Engineering**
   - ✅ Often more impactful than algorithm choice
   - ✅ Domain knowledge is key
   - ✅ Create interactions, ratios, aggregations
   - **Pro Tip:** Spend 80% time on features, 20% on algorithms

### **2. Feature Selection**
   - ✅ Reduces overfitting
   - ✅ Faster training and inference
   - ✅ Better interpretability
   - **Methods:** Filter, Wrapper, Embedded

### **3. Model Stacking**
   - ✅ Combines diverse models
   - ✅ Learns optimal combination
   - ✅ 1-2% improvement (huge in production!)
   - **Use when:** Maximum accuracy needed (Kaggle, critical systems)

### **4. ML Pipelines**
   - ✅ Prevents data leakage
   - ✅ Reproducible and maintainable
   - ✅ Production-ready
   - **Essential for:** All production ML systems

**🌟 Real-World Applications (2024-2025):**

- **Kaggle Competitions**: Stacking + feature engineering = winning solutions
- **Production ML**: Pipelines are mandatory for deployment
- **Agentic AI**: Intent classification, query routing, decision-making
- **RAG Systems**: Combining multiple retrieval signals via stacking
- **Hybrid AI**: Combining traditional ML + neural networks
- **MLOps**: Pipelines enable monitoring, A/B testing, retraining

**Production Checklist:**
- ✅ Feature engineering pipeline
- ✅ Proper train/test split (no leakage!)
- ✅ Cross-validation for robust evaluation
- ✅ Ensemble or stacking for accuracy
- ✅ sklearn Pipeline for deployment
- ✅ Model versioning and monitoring

## 🚀 Next Steps & Career Path

**You've completed Week 10! You now know:**
- Ensemble methods (Bagging, Boosting, Random Forests)
- Gradient boosting (XGBoost, LightGBM, CatBoost)
- Advanced techniques (Feature engineering, Stacking, Pipelines)

**Practice Projects:**
1. **Kaggle Competition**: Apply stacking to a real competition
2. **Build RAG System**: Use ensemble for document ranking
3. **Agentic AI**: Build multi-agent routing system

**Real-World Applications:**
- **Data Science Role**: You can now build production ML systems
- **ML Engineer**: You understand deployment pipelines
- **AI Engineer**: You can combine ML with LLMs/Transformers

**Continue Learning:**
- **Deep Learning**: Neural networks, Transformers, LLMs
- **MLOps**: Model deployment, monitoring, retraining
- **Specialized AI**: Computer Vision, NLP, Reinforcement Learning

---

**🎉 Congratulations!** You now have professional-level ML skills:
- Top Kaggle techniques
- Production ML patterns
- Modern AI system building

**💬 The Journey:**
- Week 1-5: Python fundamentals
- Week 6-8: Machine learning basics
- Week 9: Model evaluation
- **Week 10**: Advanced ML (you are here!)
- Next: Deep Learning, Transformers, and beyond!

---

*Remember: Modern AI = Traditional ML + Deep Learning. You now master the ML side. The combination of both makes you a complete AI engineer in 2024-2025!* 🌟

**Keep building, keep learning!** 🚀